In [1]:
import pandas as pd
import re

In [2]:
conserved_ptm_map_df = pd.read_csv('Data/Mapped_PTMs.csv')
conserved_ptm_map_df

,Unnamed: 0,Organism,Gene,Peptide,Observed Modifications,best_locs,start_position,end_position,a_start_position,a_end_position,alignment_length,fasta_id,a_best_locs_positions
0,0,Paracoccus_denitrificans,accA,APAEAIAAVGEAIAAMLGELAGK,Oxidation,APAEAIAAVGEAIAAMLGELAGK,273,295,719,776,962,Paracoccus_denitrificans(accA),[]
1,1,Rhizobium_radiobacter,accA,VYMLEHAIYSVISPEGAASILWR,Oxidation,VYMLEHaIYSVISPEGAASILWR,213,235,555,622,962,Rhizobium_radiobacter(accA),[561]
2,2,Bacillus_subtilis,accB,AIDESTIDEFVYENEGVSLK,Cation:Fe[III],AIDestideFVYENEGVSLK,12,31,39,58,195,Bacillus_subtilis(accB),"[42, 43, 44, 45, 46, 47]"
3,3,Stigmatella_aurantiaca,accB,RGHGPAPTIVHAAPVSPSVSPAPAVEYAAPAPAR,Cation:Fe[III],RGHGPAPTIVHAAPVSpsvspapaveYAAPAPAR,61,94,62,95,195,Stigmatella_aurantiaca(accB),"[78, 79, 80, 81, 82, 83, 84, 85, 86, 87]"
4,4,Chryseobacterium_indologenes,accB,ILVDDATPVEYDQPLFLVDPS,Ammonium,ILVDDATPVEYDQplfLVDPS,139,159,174,194,195,Chryseobacterium_indologenes(accB),"[187, 188, 189]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19619,19619,Rhodopseudomonas_palustris,zwf,DHIEHVQITVEEK,Methyl,DHIEHvQITVEEK,217,229,246,258,557,Rhodopseudomonas_palustris(zwf),[251]
19620,19620,Rhodopseudomonas_palustris,zwf,DHIEHVQITVEEK,Methyl,DHIEHvQITVEEK,217,229,246,258,557,Rhodopseudomonas_palustris(zwf),[251]
19621,19621,Rhodopseudomonas_palustris,zwf,DHIEHVQITVEEK,Methyl,DHIEHvQITVEEK,217,229,246,258,557,Rhodopseudomonas_palustris(zwf),[251]
19622,19622,Rhodopseudomonas_palustris,zwf,FANGMFEPIWNR,Carboxy->Thiocarboxy,FaNGMFEPIWNR,205,216,234,245,557,Rhodopseudomonas_palustris(zwf),[235]


In [3]:
# range of conserved PTMs

conserved_ptm_rows = []
for gene in pd.unique(conserved_ptm_map_df.Gene): #loop through data by gene
    gene_df = conserved_ptm_map_df[conserved_ptm_map_df.Gene == gene]
    gene_length = list(gene_df.alignment_length)[0]
    for modification in pd.unique(gene_df['Observed Modifications']): # loop through each modification for the gene
        conserved_ptms = {}
        modifications = {}
        mod_df = gene_df[gene_df['Observed Modifications'] == modification]
        for i in range(0, gene_length + 1): #make a map of each residue on the gene
            modifications[i] =  [set(), set()]
        for index, row in mod_df.iterrows():
            organism = row['Organism']
            fasta_id = row['fasta_id']
            a_start = row['a_start_position']
            a_end = row['a_end_position'] + 1
            for i in range(a_start, a_end): # add organism to each residue it is modified in 
                modifications[i][0].add(organism)
                modifications[i][1].add(fasta_id)
        for i in range(0, len(modifications)): # get rid of all residues with less than 2 organisms (not conserved)
            if len(modifications[i][0]) < 2:
                del modifications[i]
        if len(modifications) > 0: # if the modification is conserved (found in more than 1 species at at least 1 residue) add the map of modified residues to the conserved ptm df
            conserved_ptms['Gene'] = gene
            conserved_ptms['Modification'] = modification
            conserved_ptms['mod_residues_dic'] = modifications
            conserved_ptm_rows.append(conserved_ptms)
conserved_ptms = pd.DataFrame(conserved_ptm_rows)
conserved_ptms.to_csv('Data/Conserved_ptms.csv')
conserved_ptms      
            
            

,Gene,Modification,mod_residues_dic
0,accB,Ammonium,"{174: [{'Chryseobacterium_indologenes', 'Citro..."
1,ackA,Carbamyl,"{400: [{'Cellvibrio_gilvus', 'Citrobacter_freu..."
2,acnA,Cation:Na,"{271: [{'Stigmatella_aurantiaca', 'Sulfobacill..."
3,acnA,glyoxalAGE,"{271: [{'Stigmatella_aurantiaca', 'Sulfobacill..."
4,acnA,Carbamyl,"{519: [{'Mycobacterium_smegmatis', 'Sulfobacil..."
...,...,...,...
978,upp,Ammonium,"{164: [{'Bacillus_subtilis', 'Sulfobacillus_th..."
979,valS,Ammonium,"{1153: [{'Bacillus_subtilis', 'Coprococcus_com..."
980,valS,Methyl:2H(3),"{1153: [{'Bacillus_subtilis', 'Coprococcus_com..."
981,ychF,Ammonia-loss,"{263: [{'Cellulophaga_baltica', 'Bacillus_subt..."


In [4]:
# same logic as above, but for best locs positions rather than peptide positions (more specific)

conserved_ptm_rows = []
for gene in pd.unique(conserved_ptm_map_df.Gene):
    gene_df = conserved_ptm_map_df[conserved_ptm_map_df.Gene == gene]
    gene_length = list(gene_df.alignment_length)[0]
    for modification in pd.unique(gene_df['Observed Modifications']):
        conserved_ptms = {}
        modifications = {}
        mod_df = gene_df[gene_df['Observed Modifications'] == modification]
        for i in range(0, gene_length + 1):
            modifications[i] =  [set(), set()]
        for index, row in mod_df.iterrows():
            organism = row['Organism']
            a_start = row['a_start_position']
            a_end = row['a_end_position'] + 1
            fasta_id  = row['fasta_id']
            # convert string to list
            a_best_locs = []
            best_locs = row['a_best_locs_positions']
            for i in re.split(',', best_locs):
                i = i.replace('[', '')
                i = i.replace(']', '')
                i = i.replace(' ', '')
                if i == '': # no best loc specified
                    a_best_locs = range(a_start, a_end)
                else:
                    i = int(i)
                    a_best_locs.append(i)
            for i in a_best_locs:
                modifications[i][0].add(organism)
                modifications[i][1].add(fasta_id)
        for i in range(0, len(modifications)):
            if len(modifications[i][0]) < 2:
                del modifications[i]
        if len(modifications) > 0:
            conserved_ptms['Gene'] = gene
            conserved_ptms['Modification'] = modification
            conserved_ptms['mod_residues_dic'] = modifications
            conserved_ptm_rows.append(conserved_ptms)
conserved_ptms = pd.DataFrame(conserved_ptm_rows)
conserved_ptms.to_csv('Data/Conserved_ptms_best_loc.csv')
conserved_ptms

,Gene,Modification,mod_residues_dic
0,accB,Ammonium,"{187: [{'Chryseobacterium_indologenes', 'Citro..."
1,ackA,Carbamyl,"{403: [{'Cellvibrio_gilvus', 'Citrobacter_freu..."
2,acnA,Carbamyl,"{522: [{'Mycobacterium_smegmatis', 'Sulfobacil..."
3,acpP,Carbamyl,"{94: [{'Delftia_acidovorans', 'Acidiphilium_cr..."
4,acpP,Acetyl:13C(2),"{155: [{'Delftia_acidovorans', 'Stigmatella_au..."
...,...,...,...
742,tyrS,Methyl,"{392: [{'Rhodopseudomonas_palustris', 'Cupriav..."
743,valS,Ammonium,"{1159: [{'Bacillus_subtilis', 'Coprococcus_com..."
744,valS,Methyl:2H(3),"{1159: [{'Bacillus_subtilis', 'Coprococcus_com..."
745,ychF,Ammonia-loss,"{263: [{'Cellulophaga_baltica', 'Bacillus_subt..."
